In [1]:
from __future__ import print_function , unicode_literals , absolute_import , division
import tensorflow as tf
import numpy as np

## 定义添加层的函数

In [2]:
def add_layer(inputs , in_size , out_size , n_layer , activation_function=None ):##第二课添加了一个参数n_layer
    
    layer_name = 'layer%s'%n_layer## 添加layer_name(第二课加)
    
    with tf.name_scope('layer_name'):##第二课将layer改为layer_name
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size , out_size]) , name='W')
            
            tf.summary.histogram(layer_name + '/weights' , Weights)## 添加historgram（第二课加）
        #tf.zeros( 维度，格式，名字)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1 , out_size]) + 0.1 , name='b' )
            
            tf.summary.histogram(layer_name + '/biases' , biases)## 添加historgram（第二课加）
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul( inputs , Weights ) , biases)
        if activation_function is None :
            outputs = Wx_plus_b
        else:
            outputs = activation_function( Wx_plus_b )
            ## 添加historgram（第二课加）
            tf.summary.histogram(layer_name + '/outputs' , outputs)
        return outputs

## 真实数据(tensorboard 第二课添加)

In [3]:
# Make up some real data
x_data = np.linspace(-1 , 1 ,300)[:, np.newaxis]
noise = np.random.normal(0 , 0.05 , x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

## 定义xs,ys用于输入数据

In [4]:
with tf.name_scope('inputs'):
    xs = tf.placeholder( tf.float32 , [ None , 1 ] , name='x_input')
    ys = tf.placeholder( tf.float32 , [ None , 1 ] , name='y_input')

## 添加层(第二课中在两个函数中添加两个参数，n_layer=1和n_layer=2)

In [5]:
l1 = add_layer( xs , 1 , 10 , n_layer=1 , activation_function=tf.nn.relu )
prediction = add_layer( l1 , 10 , 1 , n_layer=2 , activation_function=None )

## 损失率

In [6]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction) , reduction_indices=[1]))
    
    
    tf.summary.scalar('loss',loss)## 添加显示至scalar(第二课加)

## 训练方式

In [7]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

## 建立会话

In [8]:
sess = tf.Session()

merged = tf.summary.merge_all()##将所有的summary合并在一起通过下方的FileWriter输出

writer = tf.summary.FileWriter("logs/" , sess.graph)

### 此处运行完之后会生成一个文件位于logs目录下；
### 进入cmd命令行模式切换到**正确的路径**下，我的logs路径是：C:\Users\24301（\logs），括号内的不需要，cd C:\Users\24301;
### 激活tensorflow
### 输入tensorboard --logdir=logs,复制生成的网址路径，改电脑名为localhost在浏览器中打开

## 初始化

In [9]:
sess.run(tf.global_variables_initializer())

In [11]:
for i in range(1000):
    sess.run(train_step , feed_dict={xs:x_data , ys:y_data})
    if i%50 == 0 :
        result = sess.run(merged , feed_dict={xs:x_data , ys:y_data})
        writer.add_summary(result , i)